In [207]:
import pandas as pd
import numpy as np

data_climate = pd.read_csv('dataset/data_climate.csv')
data_management = pd.read_csv('dataset/data_management.csv')
data_traits = pd.read_csv('dataset/data_traits.csv')
data_trials = pd.read_csv('dataset/data_trials.csv')

## Data Preprocessing

In [208]:
data_traits.head(3)

,experiment_id,management,crop_type,crop_season,plant_family,species,species_mix,cultivar,cultivar_mix,density_relative,density_factor,sowing_date,harvest_date,N_amount,measurement_date,variable,value
0,Angers_brainsurlauthion_2011,M1,SC,winter,cereal,wheat_aestivum,wheat_aestivum,cezanne,cezanne,1.0,1,2010-10-29,2011-07-06,140,2011-03-23,biomass_shoot,0.913225
1,Angers_brainsurlauthion_2011,M1,SC,winter,cereal,wheat_aestivum,wheat_aestivum,cezanne,cezanne,1.0,1,2010-10-29,2011-07-06,140,2011-03-28,height,0.324667
2,Angers_brainsurlauthion_2011,M1,SC,winter,cereal,wheat_aestivum,wheat_aestivum,cezanne,cezanne,1.0,1,2010-10-29,2011-07-06,140,2011-05-12,height,0.924000


In [209]:
data_traits[['measurement_date', 'harvest_date', 'sowing_date']] = data_traits[
    ['measurement_date', 'harvest_date', 'sowing_date']
].apply(lambda col: pd.to_datetime(col).dt.normalize())

In [210]:
data_traits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34737 entries, 0 to 34736
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   experiment_id     34737 non-null  object        
 1   management        34737 non-null  object        
 2   crop_type         34737 non-null  object        
 3   crop_season       34737 non-null  object        
 4   plant_family      34737 non-null  object        
 5   species           34737 non-null  object        
 6   species_mix       34737 non-null  object        
 7   cultivar          34737 non-null  object        
 8   cultivar_mix      34737 non-null  object        
 9   density_relative  34737 non-null  float64       
 10  density_factor    34737 non-null  object        
 11  sowing_date       34737 non-null  datetime64[ns]
 12  harvest_date      34737 non-null  datetime64[ns]
 13  N_amount          34737 non-null  int64         
 14  measurement_date  3473

In [211]:
# merge values that has plant_density value before harvest and a biomass_seed(yield) value on harvest

# stable ID columns for merging (exclude crop_season, etc.)
merge_keys = [
    'experiment_id', 'management', 'crop_type', 'crop_season',
    'plant_family', 'species', 'species_mix',
    'sowing_date', 'harvest_date'
]

# Get biomass_seed at harvest
biomass_seed_df = data_traits[
    (data_traits['variable'] == 'biomass_seed') &
    (data_traits['measurement_date'] == data_traits['harvest_date'])
][merge_keys + ['value']].rename(columns={'value': 'biomass_seed'})

# Get plant_density before harvest
plant_density_df = data_traits[
    (data_traits['variable'] == 'plant_density') &
    (data_traits['measurement_date'] < data_traits['harvest_date'])
][merge_keys + ['value']].rename(columns={'value': 'plant_density'})

# Merge biomass and density on stable keys
merged_df = pd.merge(biomass_seed_df, plant_density_df, on=merge_keys, how='inner')

# Now join back excluded features from original data_traits (dropping duplicates)
extra_features = data_traits[merge_keys + ['density_relative', 'density_factor', 'N_amount', 'cultivar', 'cultivar_mix']].drop_duplicates()

merged_df = pd.merge(merged_df, extra_features, on=merge_keys, how='left')

In [212]:
merged_df.head(2).T

,0,1
experiment_id,Angers_fnams_2002,Angers_fnams_2002
management,M1,M2
crop_type,SC,SC
crop_season,spring,spring
plant_family,cereal,legume
species,barley,pea
species_mix,barley,pea
sowing_date,2002-03-28 00:00:00,2002-03-28 00:00:00
harvest_date,2002-07-08 00:00:00,2002-07-08 00:00:00
biomass_seed,3.9,6.2


In [213]:
intercrops = [col for col in data_traits['species_mix'].unique() if '_' in col]

merged_df = merged_df[merged_df['species_mix'].isin(intercrops)]

In [214]:
merged_df

,experiment_id,management,crop_type,crop_season,plant_family,species,species_mix,sowing_date,harvest_date,biomass_seed,plant_density,density_relative,density_factor,N_amount,cultivar,cultivar_mix
2,Angers_fnams_2002,M3,IC,spring,cereal,barley,barley_pea,2002-03-28,2002-07-08,1.300000,33.000000,0.33,0.33_1,0,scarlett,scarlett_baccara
3,Angers_fnams_2002,M3,IC,spring,legume,pea,barley_pea,2002-03-28,2002-07-08,3.400000,54.000000,1.00,0.33_1,0,baccara,scarlett_baccara
8,Angers_fnams_2003,M5,IC,spring,cereal,barley,barley_pea,2003-03-12,2003-07-08,1.968870,100.000000,0.50,0.5_0.5,0,scarlett,scarlett_baccara
9,Angers_fnams_2003,M5,IC,spring,legume,pea,barley_pea,2003-03-12,2003-07-08,4.321120,45.000000,0.50,0.5_0.5,0,baccara,scarlett_baccara
10,Angers_fnams_2003,M6,IC,spring,cereal,barley,barley_pea,2003-03-12,2003-07-08,4.490746,127.000000,0.50,0.5_0.5,130,scarlett,scarlett_baccara
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,Sanmarcoargentano_sanmarcoargentano_2004,M3,IC,spring,legume,pea,barley_pea,2004-03-15,2004-06-25,2.122396,91.250000,0.50,0.5_0.5,0,baccara,scarlett_baccara
790,Taastrup_taastrup_2003,M3,IC,spring,cereal,barley,barley_pea,2003-04-04,2003-07-26,1.929750,115.833333,0.50,0.5_0.5,0,scarlett,scarlett_baccara
791,Taastrup_taastrup_2003,M3,IC,spring,legume,pea,barley_pea,2003-04-04,2003-07-26,1.418750,38.333333,0.50,0.5_0.5,0,baccara,scarlett_baccara
792,Taastrup_taastrup_2003,M4,IC,spring,cereal,barley,barley_pea,2003-04-04,2003-07-26,1.627250,122.500000,0.50,0.5_1,0,scarlett,scarlett_baccara


In [215]:
data_management.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   experiment_id           915 non-null    object 
 1   management              915 non-null    object 
 2   crop_type               915 non-null    object 
 3   crop_season             915 non-null    object 
 4   species                 915 non-null    object 
 5   species_mix             915 non-null    object 
 6   cultivar                915 non-null    object 
 7   cultivar_mix            915 non-null    object 
 8   density_level           915 non-null    float64
 9   density_relative        915 non-null    float64
 10  density_factor          915 non-null    object 
 11  interrow                858 non-null    float64
 12  mixing_pattern          616 non-null    object 
 13  code_row                118 non-null    object 
 14  mixture_design          616 non-null    ob

In [216]:
data_management.head(2).T

,0,1
experiment_id,Angers_brainsurlauthion_2011,Angers_brainsurlauthion_2011
management,M1,M2
crop_type,SC,SC
crop_season,winter,winter
species,wheat_aestivum,pea
species_mix,wheat_aestivum,pea
cultivar,cezanne,aoph10
cultivar_mix,cezanne,aoph10
density_level,250.0,80.0
density_relative,1.0,1.0


In [217]:
data_management[['harvest_date', 'sowing_date']] = data_management[
    ['harvest_date', 'sowing_date']
].apply(lambda col: pd.to_datetime(col).dt.normalize())

In [218]:
# Merge processed data_traits with data_management
merge_keys = [
    'experiment_id', 'management', 'crop_type', 'crop_season', 
    'species', 'species_mix',
    'sowing_date', 'harvest_date'
]

traits_management = pd.merge(
    merged_df,
    data_management,
    on=merge_keys,  # Ensure these columns exist in both DataFrames
    how='left',
    suffixes=('', '_management')
)

In [219]:
traits_management.head(2).T

,0,1
experiment_id,Angers_fnams_2002,Angers_fnams_2002
management,M3,M3
crop_type,IC,IC
crop_season,spring,spring
plant_family,cereal,legume
species,barley,pea
species_mix,barley_pea,barley_pea
sowing_date,2002-03-28 00:00:00,2002-03-28 00:00:00
harvest_date,2002-07-08 00:00:00,2002-07-08 00:00:00
biomass_seed,1.3,3.4


In [220]:
data_trials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country            37 non-null     object 
 1   site               37 non-null     object 
 2   plot_name          37 non-null     object 
 3   year               37 non-null     int64  
 4   experiment_id      37 non-null     object 
 5   plot_lat           37 non-null     float64
 6   plot_long          37 non-null     float64
 7   soil_awc           8 non-null      float64
 8   soil_clay          34 non-null     float64
 9   soil_sand          34 non-null     float64
 10  soil_silt          34 non-null     float64
 11  soil_pH            30 non-null     float64
 12  previous_crop      24 non-null     object 
 13  nitrogen_residual  18 non-null     float64
dtypes: float64(8), int64(1), object(5)
memory usage: 4.2+ KB


In [221]:
data_trials.head(2)

,country,site,plot_name,year,experiment_id,plot_lat,plot_long,soil_awc,soil_clay,soil_sand,soil_silt,soil_pH,previous_crop,nitrogen_residual
0,France,Angers,brainsurlauthion,2011,Angers_brainsurlauthion_2011,47.433333,-0.433333,NaN,0.51,0.23,0.26,NaN,NaN,NaN
1,France,Angers,fnams,2002,Angers_fnams_2002,47.450000,-0.400000,70.0,NaN,NaN,NaN,NaN,mustard,22.0


In [222]:
merge_keys = ['experiment_id']

traits_management_trials = pd.merge(
    traits_management,
    data_trials,
    on=merge_keys,
    how='left',
    suffixes=('', '_trials')
)

In [223]:
traits_management_trials.columns

Index(['experiment_id', 'management', 'crop_type', 'crop_season',
       'plant_family', 'species', 'species_mix', 'sowing_date', 'harvest_date',
       'biomass_seed', 'plant_density', 'density_relative', 'density_factor',
       'N_amount', 'cultivar', 'cultivar_mix', 'cultivar_management',
       'cultivar_mix_management', 'density_level',
       'density_relative_management', 'density_factor_management', 'interrow',
       'mixing_pattern', 'code_row', 'mixture_design', 'N_amount_management',
       'N_fertilisation_number', 'N_fertilisation_dose_1',
       'N_fertilisation_dose_2', 'N_fertilisation_dose_3',
       'N_fertilisation_date_1', 'N_fertilisation_date_2',
       'N_fertilisation_date_3', 'N_fertilisation_type', 'N_fertiliser',
       'irrigation_amount', 'P_amount', 'pest_control', 'country', 'site',
       'plot_name', 'year', 'plot_lat', 'plot_long', 'soil_awc', 'soil_clay',
       'soil_sand', 'soil_silt', 'soil_pH', 'previous_crop',
       'nitrogen_residual'],
     

In [224]:
data_climate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27024 entries, 0 to 27023
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   experiment_id      27024 non-null  object 
 1   date               27024 non-null  object 
 2   T2M                27024 non-null  float64
 3   T2M_MIN            27024 non-null  float64
 4   T2M_MAX            27024 non-null  float64
 5   PRECTOTCORR        27024 non-null  float64
 6   ALLSKY_SFC_SW_DWN  27024 non-null  float64
 7   RH2M               27024 non-null  float64
 8   PET                27024 non-null  float64
dtypes: float64(7), object(2)
memory usage: 1.9+ MB


In [225]:
# Example: Growing season aggregation
climate_agg = data_climate.groupby('experiment_id').agg({
    'T2M': ['mean', 'max'],                          # Mean and peak temperatures
    'T2M_MAX': lambda x: (x > 30).sum(),             # Heat stress days
    'PRECTOTCORR': 'sum',                            # Total precipitation
    'ALLSKY_SFC_SW_DWN': 'sum',                      # Total solar radiation
    'PET': 'mean',                                   # Average water demand
    'RH2M': lambda x: (x > 80).sum()                 # High humidity days
}).reset_index()

# Rename columns for clarity
climate_agg.columns = [
    'experiment_id',
    'mean_temperature', 'max_temperature',
    'heat_stress_days', 'total_precipitation',
    'total_solar_radiation', 'mean_PET',
    'high_humidity_days'
]

In [226]:
climate_agg

,experiment_id,mean_temperature,max_temperature,heat_stress_days,total_precipitation,total_solar_radiation,mean_PET,high_humidity_days
0,Angers_brainsurlauthion_2011,11.361178,26.01,6,1221.79,9468.26,2.301465,379
1,Angers_fnams_2002,11.349959,24.19,0,1643.07,8682.13,1.971932,548
2,Angers_fnams_2003,11.678068,28.21,15,1451.31,9007.49,2.153154,462
3,Angers_jailliere_2007,11.828959,27.40,11,1636.91,9026.10,2.145767,500
4,Angers_jailliere_2008,11.295335,21.65,0,1592.27,9015.77,1.980146,540
5,Angers_thorigne_2003,11.678068,28.21,15,1451.31,9007.49,2.150353,462
6,Angers_thorigne_2004,11.533639,28.21,20,1255.58,9268.02,2.254875,416
7,Angers_thorigne_2006,11.940425,27.90,34,1326.96,9298.84,2.390584,395
8,Angers_thorigne_2007,11.820562,27.90,18,1532.13,9026.10,2.193891,463
9,Angers_thorigne_2008,11.276908,22.37,1,1503.56,9015.77,2.034235,493


In [227]:
merge_keys = ['experiment_id']

final_df = pd.merge(
    traits_management_trials,
    climate_agg,
    on=['experiment_id'],
    how='left',
    suffixes=('', '_climate')
)

In [228]:
final_df.columns

Index(['experiment_id', 'management', 'crop_type', 'crop_season',
       'plant_family', 'species', 'species_mix', 'sowing_date', 'harvest_date',
       'biomass_seed', 'plant_density', 'density_relative', 'density_factor',
       'N_amount', 'cultivar', 'cultivar_mix', 'cultivar_management',
       'cultivar_mix_management', 'density_level',
       'density_relative_management', 'density_factor_management', 'interrow',
       'mixing_pattern', 'code_row', 'mixture_design', 'N_amount_management',
       'N_fertilisation_number', 'N_fertilisation_dose_1',
       'N_fertilisation_dose_2', 'N_fertilisation_dose_3',
       'N_fertilisation_date_1', 'N_fertilisation_date_2',
       'N_fertilisation_date_3', 'N_fertilisation_type', 'N_fertiliser',
       'irrigation_amount', 'P_amount', 'pest_control', 'country', 'site',
       'plot_name', 'year', 'plot_lat', 'plot_long', 'soil_awc', 'soil_clay',
       'soil_sand', 'soil_silt', 'soil_pH', 'previous_crop',
       'nitrogen_residual', 'mean_

In [230]:
final_df

,experiment_id,management,crop_type,crop_season,plant_family,species,species_mix,sowing_date,harvest_date,biomass_seed,...,soil_pH,previous_crop,nitrogen_residual,mean_temperature,max_temperature,heat_stress_days,total_precipitation,total_solar_radiation,mean_PET,high_humidity_days
0,Angers_fnams_2002,M3,IC,spring,cereal,barley,barley_pea,2002-03-28,2002-07-08,1.300000,...,NaN,mustard,22.0,11.349959,24.19,0,1643.07,8682.13,1.971932,548
1,Angers_fnams_2002,M3,IC,spring,legume,pea,barley_pea,2002-03-28,2002-07-08,3.400000,...,NaN,mustard,22.0,11.349959,24.19,0,1643.07,8682.13,1.971932,548
2,Angers_fnams_2003,M5,IC,spring,cereal,barley,barley_pea,2003-03-12,2003-07-08,1.968870,...,NaN,barley,71.0,11.678068,28.21,15,1451.31,9007.49,2.153154,462
3,Angers_fnams_2003,M5,IC,spring,legume,pea,barley_pea,2003-03-12,2003-07-08,4.321120,...,NaN,barley,71.0,11.678068,28.21,15,1451.31,9007.49,2.153154,462
4,Angers_fnams_2003,M6,IC,spring,cereal,barley,barley_pea,2003-03-12,2003-07-08,4.490746,...,NaN,barley,71.0,11.678068,28.21,15,1451.31,9007.49,2.153154,462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,Sanmarcoargentano_sanmarcoargentano_2004,M3,IC,spring,legume,pea,barley_pea,2004-03-15,2004-06-25,2.122396,...,7.7,durum_wheat,NaN,14.823707,29.68,105,2123.88,11463.44,2.841437,336
628,Taastrup_taastrup_2003,M3,IC,spring,cereal,barley,barley_pea,2003-04-04,2003-07-26,1.929750,...,6.6,winter_wheat,NaN,8.734178,22.40,0,1440.75,7729.00,1.592443,579
629,Taastrup_taastrup_2003,M3,IC,spring,legume,pea,barley_pea,2003-04-04,2003-07-26,1.418750,...,6.6,winter_wheat,NaN,8.734178,22.40,0,1440.75,7729.00,1.592443,579
630,Taastrup_taastrup_2003,M4,IC,spring,cereal,barley,barley_pea,2003-04-04,2003-07-26,1.627250,...,6.6,winter_wheat,NaN,8.734178,22.40,0,1440.75,7729.00,1.592443,579


In [231]:
final_df.to_csv('merged_df')